In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LIONESS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [17]:
df = pd.read_csv('Day4_Fake_News_Data.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
df['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [18]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
df = df.fillna('')

In [20]:
df['content'] = df['author']+' '+df['title']

In [21]:
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [22]:
X = df.drop(columns='label', axis=1)
Y = df['label']

In [23]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [24]:
port_stem = PorterStemmer()

In [25]:
def stemming(content):
    stemmed_content = re.sub('[˄a-zA-Z],',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [26]:
df['content'] = df['content'].apply(stemming)

In [27]:
print(df['content'])

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinto big woma...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedul matc...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nat russia hold parallel exercis b...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [28]:
X = df['content'].values
Y = df['label'].values

In [29]:
print(X)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinto big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nat russia hold parallel exercis balkan'
 'david swanson keep f-35 aliv']


In [30]:
print(Y)

[1 0 1 ... 0 1 1]


In [31]:
Y.shape

(20800,)

In [32]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [33]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 217863 stored elements and shape (20800, 22459)>
  Coords	Values
  (0, 820)	0.36436410155794613
  (0, 3498)	0.3414174549077228
  (0, 4072)	0.2271602610877475
  (0, 4909)	0.3301866134165259
  (0, 5146)	0.24982904839258457
  (0, 5412)	0.2744733135119046
  (0, 6629)	0.21389213615736222
  (0, 9118)	0.2051808187185159
  (0, 9999)	0.22739333087388314
  (0, 11085)	0.26867913406443367
  (0, 11423)	0.3335834569339999
  (0, 16993)	0.23657500428226982
  (0, 19690)	0.2707524924236901
  (1, 2311)	0.2814241402995684
  (1, 2781)	0.14844508332853032
  (1, 3182)	0.36778732168045225
  (1, 3895)	0.3190386473446403
  (1, 4874)	0.25219472647304686
  (1, 7318)	0.6919765846126547
  (1, 8889)	0.1902359272858601
  (1, 21040)	0.28824964225569444
  (2, 4053)	0.31659503087284113
  (2, 4289)	0.46008093017866314
  (2, 7181)	0.39364329037139567
  (2, 7888)	0.34593851964036826
  :	:
  (20797, 9163)	0.21835041033036617
  (20797, 10761)	0.22033598069604873
 

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [35]:
model = LogisticRegression()

In [36]:
model.fit(X_train, Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [37]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [38]:
print("Accuracy score of the trained data: ", training_data_accuracy)

Accuracy score of the trained data:  0.9868990384615385


In [39]:
X_test_prediction = model.predict(X_test)
test_accuracy_score = accuracy_score(X_test_prediction, Y_test)

In [40]:
print("Accuracy score of the test data: ", test_accuracy_score)

Accuracy score of the test data:  0.9776442307692308


In [45]:
X_new = X_test[100]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print("The news is Real")
else:
    print("The news is Fake")

[0]
The news is Real


In [46]:
print(Y_test[100])

0
